In [5]:
import sys 
import dimod
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import dwave_networkx as dnx
import dwave.inspector
import dwavebinarycsp
import pandas as pd
from itertools import permutations 
import operator
from operator import itemgetter
from dimod.reference.samplers import ExactSolver
from dwave.system.samplers import DWaveSampler
from dwave.system.composites import EmbeddingComposite
from dwave.system import DWaveSampler, EmbeddingComposite
from dwave_networkx.utils import binary_quadratic_model_sampler
from dwave.system import TilingComposite
from minorminer import find_embedding

In [6]:
from __future__ import division

import itertools

from collections import defaultdict

from dwave_networkx.utils import binary_quadratic_model_sampler

__all__ = ["traveling_salesperson",
           "traveling_salesperson_qubo",
           "traveling_salesman",
           "traveling_salesman_qubo",
           "is_hamiltonian_path",
           ]


In [7]:
@binary_quadratic_model_sampler(1)
def traveling_salesperson(G, sampler=None, lagrange=None, weight='weight',
                          start=None, **sampler_args):
    
    # Get a QUBO representation of the problem
    Q = traveling_salesperson_qubo(G, lagrange, weight)
    print(Q)




    # use the sampler to find low energy states
    response = sampler.sample_qubo(Q, **sampler_args)
    dwave.inspector.show(Q, response)

    sample = response.first.sample

    route = [None]*len(G)
    for (city, time), val in sample.items():
        if val:
            route[time] = city

    if start is not None and route[0] != start:
        # rotate to put the start in front
        idx = route.index(start)
        route = route[idx:] + route[:idx]

    return route



traveling_salesman = traveling_salesperson


def traveling_salesperson_qubo(G, lagrange=None, weight='weight'):
    
    
    N = G.number_of_nodes()

    if lagrange is None:
        # If no lagrange parameter provided, set to 'average' tour length.
        # Usually a good estimate for a lagrange parameter is between 75-150%
        # of the objective function value, so we come up with an estimate for 
        # tour length and use that.
        if G.number_of_edges()>0:
            lagrange = G.size(weight=weight)*G.number_of_nodes()/G.number_of_edges()
        else:
            lagrange = 2

    # some input checking
    if N in (1, 2) or len(G.edges) != N*(N-1)//2:
        msg = "graph must be a complete graph with at least 3 nodes or empty"
        raise ValueError(msg)

    # Creating the QUBO
    Q = defaultdict(float)

    # Constraint that each row has exactly one 1
    for node in G:
        for pos_1 in range(N):
            Q[((node, pos_1), (node, pos_1))] -= lagrange
            for pos_2 in range(pos_1+1, N):
                Q[((node, pos_1), (node, pos_2))] += 2.0*lagrange

    # Constraint that each col has exactly one 1
    for pos in range(N):
        for node_1 in G:
            Q[((node_1, pos), (node_1, pos))] -= lagrange
            for node_2 in set(G)-{node_1}:
                # QUBO coefficient is 2*lagrange, but we are placing this value 
                # above *and* below the diagonal, so we put half in each position.
                Q[((node_1, pos), (node_2, pos))] += lagrange

    # Objective that minimizes distance
    for u, v in itertools.combinations(G.nodes, 2):
        for pos in range(N):
            nextpos = (pos + 1) % N

            # going from u -> v
            Q[((u, pos), (v, nextpos))] += G[u][v][weight]

            # going from v -> u
            Q[((v, pos), (u, nextpos))] += G[u][v][weight]

    return Q



traveling_salesman_qubo = traveling_salesperson_qubo


def is_hamiltonian_path(G, route):

    return (set(route) == set(G))

In [8]:
dwave_sampler = EmbeddingComposite(DWaveSampler(endpoint='https://cloud.dwavesys.com/sapi',        token='DEV-d9500e99062cdd39d17bc2f4a0993eb1cae4e053', 
    solver='Advantage_system6.1'))

G = nx.complete_graph(7)
original = nx.to_numpy_array(G, dtype=int)
'''gr = nx.to_numpy_array(G, dtype=int)
temp = gr
    
def bit_flip():
    for i in range(len(gr)):
        for j in range(len(gr)):
            if temp[i,j] == 0:
                temp[i,j] = 1
            else:
                temp[i,j] = 0
            grflipped = temp
    return(temp)
    
inverted = bit_flip()'''
qubomatrix = 1*original 
qubograph = nx.to_networkx_graph(qubomatrix)

traveling_salesperson(G= qubograph, sampler = dwave_sampler, weight = 'weight')


defaultdict(<class 'float'>, {((0, 0), (0, 0)): -14.0, ((0, 0), (0, 1)): 14.0, ((0, 0), (0, 2)): 14.0, ((0, 0), (0, 3)): 14.0, ((0, 0), (0, 4)): 14.0, ((0, 0), (0, 5)): 14.0, ((0, 0), (0, 6)): 14.0, ((0, 1), (0, 1)): -14.0, ((0, 1), (0, 2)): 14.0, ((0, 1), (0, 3)): 14.0, ((0, 1), (0, 4)): 14.0, ((0, 1), (0, 5)): 14.0, ((0, 1), (0, 6)): 14.0, ((0, 2), (0, 2)): -14.0, ((0, 2), (0, 3)): 14.0, ((0, 2), (0, 4)): 14.0, ((0, 2), (0, 5)): 14.0, ((0, 2), (0, 6)): 14.0, ((0, 3), (0, 3)): -14.0, ((0, 3), (0, 4)): 14.0, ((0, 3), (0, 5)): 14.0, ((0, 3), (0, 6)): 14.0, ((0, 4), (0, 4)): -14.0, ((0, 4), (0, 5)): 14.0, ((0, 4), (0, 6)): 14.0, ((0, 5), (0, 5)): -14.0, ((0, 5), (0, 6)): 14.0, ((0, 6), (0, 6)): -14.0, ((1, 0), (1, 0)): -14.0, ((1, 0), (1, 1)): 14.0, ((1, 0), (1, 2)): 14.0, ((1, 0), (1, 3)): 14.0, ((1, 0), (1, 4)): 14.0, ((1, 0), (1, 5)): 14.0, ((1, 0), (1, 6)): 14.0, ((1, 1), (1, 1)): -14.0, ((1, 1), (1, 2)): 14.0, ((1, 1), (1, 3)): 14.0, ((1, 1), (1, 4)): 14.0, ((1, 1), (1, 5)): 14.0, (

[3, 5, 5, 5, 1, 6, 1]